In [1]:
import os
%pwd


'/home/ml01/MLOps/hugginface_project/Text-Summarizer-MLOPS/research'

In [2]:
os.chdir("../")
%pwd

'/home/ml01/MLOps/hugginface_project/Text-Summarizer-MLOPS'

## Basic configuration


In [3]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file: Path 
    unzip_dir: Path 


## Configuration Updates

In [4]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_path= CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


## Components

In [5]:
import os
import urllib.request as request
import zipfile
from src.text_summarizer.logging import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url = self.config.source_url, filename= self.config.local_data_file)
            logger.info(f"{self.config.local_data_file} is downloaded")
        else:
            logger.info(f"{self.config.local_data_file} already exists")

    def extract_zip_file(self):
        """
        zip_file_path : str
        Extracts the zip file into the data directory
        Function returns None
        """ 
        unzip_path = self.config.unzip_dir 
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


            

In [6]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config = data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-06-25 17:11:21,659: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-25 17:11:21,660: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-25 17:11:21,661: INFO: common: created directory at: artifacts]
[2025-06-25 17:11:21,661: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-25 17:11:21,662: INFO: 942659643: artifacts/data_ingestion/data.zip already exists]
